# Justin Zarkovacki 2/15/2023
# Transfer Learning KMNIST -> K-49

# Prepare imports

In [3]:
import numpy as np
import optuna
import sys
import os

from matplotlib import pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense
from keras import models, backend as K
from tensorflow.keras import models

print("Done!")

Done!


# Function Definitions and Variables

In [4]:
epochs = 12
img_rows, img_cols = 28, 28  # Image dimensions

def load(f):
    return np.load(f)['arr_0']
    
def initialize_data(train_im_file, test_im_file, train_lb_file, test_lb_file):
    train_images = load(train_im_file)
    test_images = load(test_im_file)
    train_labels = load(train_lb_file)
    test_labels = load(test_lb_file)
    
    if K.image_data_format() == 'channels_first':
        train_images = train_images.reshape(train_images.shape[0], 1, img_rows, img_cols)
        test_images = test_images.reshape(test_images.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, 1)
        test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    train_images = train_images.astype('float32')
    test_images = test_images.astype('float32')
    train_images /= 255
    test_images /= 255
    print('{} train samples, {} test samples'.format(len(train_images), len(test_images)))
    
    return tuple([train_images, test_images, train_labels, test_labels, input_shape])

# Helper to create the graphics
def create_visuals(graph_title, model_hist, test_images, test_labels):
    accuracy_data = model_hist.history['accuracy']
    val_accuracy_data = model_hist.history['val_accuracy']

    lower_bound = min(min(accuracy_data), min(val_accuracy_data))

    plt.plot(accuracy_data, label='Train Accuracy')
    plt.plot(val_accuracy_data, label = 'Validation Accuracy')

    plt.title(graph_title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([lower_bound - 0.01, 1])
    plt.legend(loc='lower right')

    print(accuracy_data[-1])
    print(val_accuracy_data[-1])
#     test_loss, test_acc = model_hist.evaluate(test_images, test_labels, verbose=2)

print("Done!")

Done!


## Load Data

In [5]:
prefix = 'C:\\Users\\LifeH\\SoftwareDevelopment\\KanjiCharacterRecognition\\'
datasets = {
    "km_tr_i" : prefix + "Datasets\\Kuzushiji-MNIST - train-imgs.npz",
    "km_te_i" : prefix + "Datasets\\Kuzushiji-MNIST - test-imgs.npz",
    "km_tr_l" : prefix + "Datasets\\Kuzushiji-MNIST - train-labels.npz",
    "km_te_l" : prefix + "Datasets\\Kuzushiji-MNIST - test-labels.npz",
    "k49_tr_i" : prefix + "Datasets\\Kuzushiji-49 - train-imgs.npz",
    "k49_te_i" : prefix + "Datasets\\Kuzushiji-49 - test-imgs.npz",
    "k49_tr_l" : prefix + "Datasets\\Kuzushiji-49 - train-labels.npz",
    "k49_te_l" : prefix + "Datasets\\Kuzushiji-49 - test-labels.npz"
}

# Load KMNIST Data
dataset = initialize_data(datasets["km_tr_i"], datasets["km_te_i"], datasets["km_tr_l"], datasets["km_te_l"])
kmnist_train_images = dataset[0]
kmnist_test_images = dataset[1]
kmnist_train_labels = dataset[2]
kmnist_test_labels  = dataset[3]
kmnist_input_shape = dataset[4]
kmnist_classes = 10
kmnist_epochs = 15

# Load Kuzushiji-49 Data
dataset = initialize_data(datasets["k49_tr_i"], datasets["k49_te_i"], datasets["k49_tr_l"], datasets["k49_te_l"])
k49_train_images = dataset[0]
k49_test_images = dataset[1]
k49_train_labels = dataset[2]
k49_test_labels  = dataset[3]
k49_input_shape = dataset[4]
k49_classes = 49
k49_epochs = 15

batches = 128
num_trials = 15

print("Done!")

60000 train samples, 10000 test samples
232365 train samples, 38547 test samples
Done!


This notebook will create an ensemble model for K49 character recognition. It wil be composed of 2 basic models, and one transfer learning model. Knowledge from KMNIST will be transfered to K49.

# Creating K49 Model 1

In [ ]:
def k49_objective1(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 32, 54)
    l2_filters = trial.suggest_int('second_layer_kernel', 20, 64)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    k49_1_design = Sequential()
    k49_1_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=k49_input_shape))
    k49_1_design.add(Dropout(dropout))
    k49_1_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=k49_input_shape))
    k49_1_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    k49_1_design.add(Flatten())
    k49_1_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    k49_1_design.add(Dense(k49_classes))

    k49_1_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(k49_1_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    k49_1_history = k49_1_design.fit(k49_train_images, k49_train_labels, epochs=k49_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(k49_test_images, k49_test_labels))

    # Important metric for optuna to optimize over
    return k49_1_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 1
fileName = prefix + 'KanjiModelDevelopmentOutputs\\K49_Model_1.txt'
orig_stdout = sys.stdout
with open(fileName, 'w') as f:
    sys.stdout = f
    k49_study1 = optuna.create_study(direction='maximize', study_name="K49-1")
    k49_study1.optimize(k49_objective1, n_trials=10)
    sys.stdout = orig_stdout
    f.close()

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{k49_study1.best_trial}\n')
for param, value in k49_study1.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=5, values=[0.9037019610404968], datetime_start=datetime.datetime(2023, 2, 22, 18, 34, 50, 657250), datetime_complete=datetime.datetime(2023, 2, 22, 18, 39, 34, 444773), params={'kernel_size': 3, 'first_layer_kernel': 54, 'second_layer_kernel': 36, 'first_layer_activation': 'relu', 'second_layer_activation': 'relu', 'dropout': 0.24272834648209657, 'average_pooling_size': 4, 'dense_layer_size': 79, 'dense_layer_activation': 'sigmoid'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=54, log=False, low=32, step=1), 'second_layer_kernel': IntDistribution(high=64, log=False, low=20, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
# Optuna kept crashing for this model. The best hyperparameters that I got are
#   listed below (even though they are not listed the same in the cell above)
kern_size	= 3
l1_filters = 42
l2_filters = 51
l1_activation = "relu"
l2_activation = "relu"
dropout	= 0.2778228681684748
average_pooling_size = 4
dense_layer_size	= 71
dense_layer_activation = "relu"

k49_1 = Sequential()
k49_1.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=k49_input_shape))
k49_1.add(Dropout(dropout))
k49_1.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=k49_input_shape))
k49_1.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
k49_1.add(Flatten())
k49_1.add(Dense(dense_layer_size, activation=dense_layer_activation))
k49_1.add(Dense(k49_classes))

k49_1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

k49_1_optuna_history = k49_1.fit(k49_train_images, k49_train_labels, epochs=k49_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(k49_test_images, k49_test_labels))
k49_1.save(prefix + 'k49_models\\k49_1.h5', save_format='h5')

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
1816/1816 [==============================] - 98s 54ms/step - loss: 0.7731 - sparse_categorical_crossentropy: 2.7256 - accuracy: 0.7996 - val_loss: 0.7443 - val_sparse_categorical_crossentropy: 2.6473 - val_accuracy: 0.8062
Epoch 2/15
1816/1816 [==============================] - 97s 54ms/step - loss: 0.3492 - sparse_categorical_crossentropy: 2.3094 - accuracy: 0.9057 - val_loss: 0.5598 - val_sparse_categorical_crossentropy: 2.4416 - val_accuracy: 0.8569
Epoch 3/15
1816/1816 [==============================] - 92s 51ms/step - loss: 0.2635 - sparse_categorical_crossentropy: 2.1781 - accuracy: 0.9284 - val_loss: 0.4946 - val_sparse_categorical_crossentropy: 2.3632 - val_accuracy: 0.8750
Epoch 4/15
1816/1816 [==============================] - 94s 52ms/step - loss: 0.2139 - sparse_categorical_crossentropy: 2.0831 - accuracy: 0.9409 - val_loss: 0.4274 - val_sparse_categorical_crossentropy: 2.2472 - val_accuracy: 0.8903
Epoch 5/15
1816/1816 [==============================] - 99s 54ms

# Creating K49 Model 2

In [ ]:
def k49_objective2(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 20, 40)
    l2_filters = trial.suggest_int('second_layer_kernel', 40, 64)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    k49_2_design = Sequential()
    k49_2_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=k49_input_shape))
    k49_2_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=k49_input_shape))
    k49_2_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    k49_2_design.add(Dropout(dropout))
    k49_2_design.add(Flatten())
    k49_2_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    k49_2_design.add(Dense(k49_classes))

    k49_2_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(k49_2_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    k49_2_history = k49_2_design.fit(k49_train_images, k49_train_labels, epochs=kmnist_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(k49_test_images, k49_test_labels))

    # Important metric for optuna to optimize over
    return k49_2_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 2
fileName = prefix + 'KanjiModelDevelopmentOutputs\\K49_Model_2.txt'
orig_stdout = sys.stdout
with open(fileName, 'w') as f:
    sys.stdout = f
    k49_study2 = optuna.create_study(direction='maximize', study_name="K49-2")
    k49_study2.optimize(k49_objective2, n_trials=10)
    sys.stdout = orig_stdout
    f.close()

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{k49_study2.best_trial}\n')
for param, value in k49_study2.best_params.items():
    print(f'Param: {param}\tValue: {value}')
  
  # 0.9286325573921204 and parameters: {'kernel_size': 3, 'first_layer_kernel': 40, 'second_layer_kernel': 57, 'first_layer_activation': 'relu', 'second_layer_activation': 'relu', 'dropout': 0.24885972769710446, 'average_pooling_size': 4, 'dense_layer_size': 75, 'dense_layer_activation': 'tanh'}. Best is trial 11 with value: 0.9286325573921204.

Best trial info:
FrozenTrial(number=2, values=[0.9010558724403381], datetime_start=datetime.datetime(2023, 2, 21, 2, 2, 24, 651315), datetime_complete=datetime.datetime(2023, 2, 21, 2, 6, 48, 559689), params={'kernel_size': 2, 'first_layer_kernel': 31, 'second_layer_kernel': 45, 'first_layer_activation': 'tanh', 'second_layer_activation': 'relu', 'dropout': 0.22502685245131498, 'average_pooling_size': 4, 'dense_layer_size': 70, 'dense_layer_activation': 'relu'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=40, log=False, low=20, step=1), 'second_layer_kernel': IntDistribution(high=64, log=False, low=40, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4, log=

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 3
l1_filters = 40
l2_filters = 57
l1_activation = "relu"
l2_activation = "relu"
dropout	= 0.24885972769710446
average_pooling_size = 4
dense_layer_size	= 75
dense_layer_activation = "tanh"

k49_2 = Sequential()
k49_2.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=k49_input_shape))
k49_2.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=k49_input_shape))
k49_2.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
k49_2.add(Dropout(dropout))
k49_2.add(Flatten())
k49_2.add(Dense(dense_layer_size, activation=dense_layer_activation))
k49_2.add(Dense(k49_classes))

k49_2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

k49_2_optuna_history = k49_2.fit(k49_train_images, k49_train_labels, epochs=k49_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(k49_test_images, k49_test_labels))
k49_2.save(prefix + 'k49_models\\k49_2.h5', save_format='h5')

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
1816/1816 [==============================] - 101s 55ms/step - loss: 0.9164 - sparse_categorical_crossentropy: 3.1510 - accuracy: 0.7728 - val_loss: 0.8247 - val_sparse_categorical_crossentropy: 3.1425 - val_accuracy: 0.7863
Epoch 2/15
1816/1816 [==============================] - 93s 51ms/step - loss: 0.4020 - sparse_categorical_crossentropy: 3.0073 - accuracy: 0.8951 - val_loss: 0.5770 - val_sparse_categorical_crossentropy: 3.0848 - val_accuracy: 0.8527
Epoch 3/15
1816/1816 [==============================] - 96s 53ms/step - loss: 0.2973 - sparse_categorical_crossentropy: 3.0028 - accuracy: 0.9213 - val_loss: 0.4807 - val_sparse_categorical_crossentropy: 3.0746 - val_accuracy: 0.8758
Epoch 4/15
1816/1816 [==============================] - 91s 50ms/step - loss: 0.2433 - sparse_categorical_crossentropy: 2.9999 - accuracy: 0.9335 - val_loss: 0.4333 - val_sparse_categorical_crossentropy: 3.0623 - val_accuracy: 0.8886
Epoch 5/15
1816/1816 [==============================] - 91s 50m

# KMNIST to K49 Transfer Learning

In [7]:
import os
import random
from skimage.transform import rescale
from skimage import io

# Loading a base model requires you pop all dimensionality flattening layers
def load_base_model(filename):
    bm = models.load_model(filename)
    bm.pop()
    bm.pop()
    bm.pop()
    bm.trainable = False
    return bm

## Creating KMNIST Base

In [ ]:
def kmnist_base_objective(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 20, 40)
    l2_filters = trial.suggest_int('second_layer_kernel', 40, 64)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    kmnist_base_design = Sequential()
    kmnist_base_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
    kmnist_base_design.add(Dropout(dropout))
    kmnist_base_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
    kmnist_base_design.add(Flatten())
    kmnist_base_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kmnist_base_design.add(Dense(kmnist_classes))

    kmnist_base_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kmnist_base_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kmnist_base_history = kmnist_base_design.fit(kmnist_train_images, kmnist_train_labels,
                    epochs=kmnist_epochs, batch_size=batches, callbacks=callback,
                    validation_data=(kmnist_test_images, kmnist_test_labels))

    # Important metric for optuna to optimize over
    return kmnist_base_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 3
fileName = prefix + 'KanjiModelDevelopmentOutputs\\KMNIST_Base_Model.txt'
orig_stdout = sys.stdout
with open(fileName, 'w') as f:
    sys.stdout = f
    kmnist_base_study = optuna.create_study(direction='maximize', study_name="KMNIST-Base")
    kmnist_base_study.optimize(kmnist_base_objective, n_trials=15)
    sys.stdout = orig_stdout
    f.close()

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{kmnist_base_study.best_trial}\n')
for param, value in kmnist_base_study.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=10, values=[0.933899998664856], datetime_start=datetime.datetime(2023, 2, 27, 16, 55, 11, 260229), datetime_complete=datetime.datetime(2023, 2, 27, 16, 56, 50, 78876), params={'kernel_size': 2, 'first_layer_kernel': 40, 'second_layer_kernel': 50, 'first_layer_activation': 'relu', 'second_layer_activation': 'tanh', 'dropout': 0.2473761636466844, 'average_pooling_size': 3, 'dense_layer_size': 80, 'dense_layer_activation': 'relu'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=40, log=False, low=20, step=1), 'second_layer_kernel': IntDistribution(high=64, log=False, low=40, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4, lo

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 2
l1_filters = 40
l2_filters = 50
l1_activation = "relu"
l2_activation = "tanh"
dropout	= 0.2473761636466844
average_pooling_size = 3
dense_layer_size	= 80
dense_layer_activation = "relu"

kmnist_base = Sequential()
kmnist_base.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
kmnist_base.add(Dropout(dropout))
kmnist_base.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
kmnist_base.add(Flatten())
kmnist_base.add(Dense(dense_layer_size, activation=dense_layer_activation))
kmnist_base.add(Dense(kmnist_classes))

kmnist_base.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
# tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

kmnist_optuna_history = kmnist_base.fit(kmnist_train_images, kmnist_train_labels, batch_size=batches, epochs=epochs,
                                callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

kmnist_base.save(prefix + 'kmnist_base.h5', save_format='h5')

Epoch 1/12
469/469 [==============================] - 23s 48ms/step - loss: 0.0187 - sparse_categorical_crossentropy: 1.1457 - accuracy: 0.9944 - val_loss: 0.3843 - val_sparse_categorical_crossentropy: 1.4009 - val_accuracy: 0.9265
Epoch 2/12
469/469 [==============================] - 22s 47ms/step - loss: 0.0157 - sparse_categorical_crossentropy: 1.1434 - accuracy: 0.9947 - val_loss: 0.4365 - val_sparse_categorical_crossentropy: 1.3413 - val_accuracy: 0.9225
Epoch 3/12
469/469 [==============================] - 21s 46ms/step - loss: 0.0146 - sparse_categorical_crossentropy: 1.1253 - accuracy: 0.9950 - val_loss: 0.4513 - val_sparse_categorical_crossentropy: 1.3972 - val_accuracy: 0.9272
Epoch 4/12
469/469 [==============================] - 22s 47ms/step - loss: 0.0121 - sparse_categorical_crossentropy: 1.1202 - accuracy: 0.9960 - val_loss: 0.4343 - val_sparse_categorical_crossentropy: 1.3633 - val_accuracy: 0.9317


## Create K-49 Top

In [7]:
def k49_top_objective(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 32, 54)
    l2_filters = trial.suggest_int('second_layer_kernel', 64, 96)
    activations = trial.suggest_categorical('activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 128)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    base_model = load_base_model(prefix + 'kmnist_base.h5')
    
    # Design model
    k49_top_design = Sequential()
    k49_top_design.add(base_model)
    k49_top_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=activations, input_shape=k49_input_shape))
    k49_top_design.add(Dropout(dropout))
    k49_top_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=activations, input_shape=k49_input_shape))
    k49_top_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    k49_top_design.add(Flatten())
    k49_top_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    k49_top_design.add(Dense(k49_classes))

    k49_top_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(k49_top_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    k49_top_history = k49_top_design.fit(k49_train_images, k49_train_labels, epochs=k49_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(k49_test_images, k49_test_labels))

    # Important metric for optuna to optimize over
    return k49_top_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 4
fileName = prefix + 'KanjiModelDevelopmentOutputs\\K49_TL_Model.txt'
orig_stdout = sys.stdout
with open(fileName, 'w') as f:
    sys.stdout = f
    k49_TL_study = optuna.create_study(direction='maximize', study_name="K49-TL-Results")
    k49_TL_study.optimize(k49_top_objective, n_trials=num_trials)
    sys.stdout = orig_stdout
    f.close()

In [10]:
# Print the info from the best trial
print(f'Best trial info:\n{k49_TL_study.best_trial}\n') 
for param, value in k49_TL_study.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=3, values=[0.9341323375701904], datetime_start=datetime.datetime(2023, 3, 1, 6, 29, 9, 881781), datetime_complete=datetime.datetime(2023, 3, 1, 6, 37, 34, 248671), params={'kernel_size': 3, 'first_layer_kernel': 49, 'second_layer_kernel': 91, 'activation': 'relu', 'dropout': 0.26155871876179343, 'average_pooling_size': 3, 'dense_layer_size': 115, 'dense_layer_activation': 'relu'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=54, log=False, low=32, step=1), 'second_layer_kernel': IntDistribution(high=96, log=False, low=64, step=1), 'activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4, log=False, low=2, step=1), 'dense_layer_size': IntDistribution(high=128, log=False, low=64, step=1), 'dense_layer_activation': CategoricalDistribution(c

In [12]:
# Optuna doesn't save the best model. You must rebuild it and save it.
base_model = load_base_model(prefix + 'kmnist_base.h5')
kern_size	= 3
l1_filters = 49
l2_filters = 91
activations = "relu"
dropout	= 0.26155871876179343
average_pooling_size = 3
dense_layer_size	= 115
dense_layer_activation = "relu"

k49_TL = Sequential()
k49_TL.add(base_model)
k49_TL.add(Conv2D(l1_filters, kernel_size=kern_size, activation=activations, input_shape=k49_input_shape))
k49_TL.add(Dropout(dropout))
k49_TL.add(Conv2D(l2_filters, kernel_size=kern_size, activation=activations, input_shape=k49_input_shape))
k49_TL.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
k49_TL.add(Flatten())
k49_TL.add(Dense(dense_layer_size, activation=dense_layer_activation))
k49_TL.add(Dense(k49_classes))

k49_TL.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [13]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

k49_TL_optuna_history = k49_TL.fit(k49_train_images, k49_train_labels, epochs=k49_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(k49_test_images, k49_test_labels))

k49_TL.save(prefix + 'k49_models\\k49_TL.h5', save_format='h5')

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
1816/1816 [==============================] - 137s 71ms/step - loss: 0.4549 - sparse_categorical_crossentropy: 2.6602 - accuracy: 0.8790 - val_loss: 0.4614 - val_sparse_categorical_crossentropy: 2.5506 - val_accuracy: 0.8789
Epoch 2/15
1816/1816 [==============================] - 123s 68ms/step - loss: 0.1911 - sparse_categorical_crossentropy: 2.3451 - accuracy: 0.9468 - val_loss: 0.3454 - val_sparse_categorical_crossentropy: 2.3977 - val_accuracy: 0.9123
Epoch 3/15
1816/1816 [==============================] - 122s 67ms/step - loss: 0.1381 - sparse_categorical_crossentropy: 2.2365 - accuracy: 0.9604 - val_loss: 0.2983 - val_sparse_categorical_crossentropy: 2.2937 - val_accuracy: 0.9221
Epoch 4/15
1816/1816 [==============================] - 120s 66ms/step - loss: 0.1068 - sparse_categorical_crossentropy: 2.1412 - accuracy: 0.9691 - val_loss: 0.2941 - val_sparse_categorical_crossentropy: 2.2212 - val_accuracy: 0.9277
Epoch 5/15
1816/1816 [==============================] - 123s